To Do:

Add spring boxes (metal/wood, one class that adds a shape of an arrow, and that's it, rest will be handled outside)

Add Tnt destruct sequence

Add protection box (and following sprite?)

Level saving

Game structure, saving, scores, etc.

Background (background should generate automatically, and should move at 1/2 speed... maybe two layers of background?)

Level generation function


Known bugs:
falling stacks of boxes ON a moving platform results in slight overlap (but nothing functionally problematic)

Credits:
“Sound effects obtained from zapsplat.com and freesound.org"
"Idea shamelessly taken from Crash Bandicoot 2: Wrath of Cortex for PS2"


In [2]:
import pygame

pygame.init()

import math
import Box
import numpy as np
from Super_Classes import Vector, Body, Shape, Looper
import Platform
from Gettables import Fruit
from random import randint
import copy
from Player import Player
from Constants import G, box_size, S
from time import sleep
from Make_Sounds import wood_bounce_sound, wood_break_sound, boom_sound, fruit_sound


                    
def resolve_fall(bod,bod2): #resolving fall for non-character objects
    if bod2.solid and bod2.pos[1]>bod.pos[1]: # bod 2 is below and solid
        dim,gap = bod.overlap_dim(bod2)
        if dim == 1 and gap>0.1:# bod2 is overlapping vertically
            bod.is_on(bod2)
            bod.pos[1] -= gap #shift bod up!
            return True # solved!
    return False




display_size = (1000,600)

gameDisplay = pygame.display.set_mode(display_size)
pygame.display.set_caption('Game!')

black = (0,0,0)
white = (255,255,255)
floor = 500

clock = pygame.time.Clock()

bounce_vel = 6.0*S

R = box_size*4
N = 600
path = [np.array([500*S + R*math.cos(2*math.pi*x/N),floor + R*math.sin(2*math.pi*x/N)]) for x in range(N)]
d1 = Platform.Moving_Platform([box_size*2,box_size/3],copy.copy(path),color=(50,50,50),line_color = None ,line_width = 2)
d2 = Platform.Moving_Platform([box_size*2,box_size/3],path[int(N/2):]+path[:int(N/2)],color=(50,50,50),line_color = (0,0,0) ,line_width = 2)
p1 = Platform.Platform([100*S,floor+50*S],[200*S,50*S])
p2 = Platform.Platform([1000*S,floor+50*S],[300*S,50*S])

p0 = Platform.Platform(*[[500*S,floor],[10*S,10*S]])
p0.solid = False
p0.corporeal = False

a = Box.Nitro(p1.pos - [-box_size*100/30,p1.size[1]+box_size])
a1 = Box.Metal(a.pos - [0,box_size*2])
a2 = Box.Nitro(a.pos - [box_size*2,0])
a3 = Box.Metal_Wood(a.pos - [box_size*2,box_size*2])
b2 = Box.Bouncey_Wood(path[int(N/2)]-[0,box_size+d1.size[1]])
b3 = Box.Metal_Wood(path[int(N/2)]-[0,box_size*3+d1.size[1]])
b4 = Box.Wood(path[int(N/2)]-[0,box_size*5+d1.size[1]])


master_platform_list = [p1,p2,d1,d2]
master_box_list = [a,a1,a2,a3,b2,b3,b4]
background_list = [p0]
foreground_list = [] # for exploding things, plus lives, score, etc

for k in range(4):
    for i in range(3):
        if i == 0:
            master_box_list.append(Box.Metal_Wood(a.pos + [600*S+box_size*2*k,-i*2*box_size]))
        else:
            master_box_list.append(Box.Wood(a.pos + [600*S+box_size*2*k,-i*2*box_size]))
for i in [0,3.0]:
    master_box_list.append(Box.Bouncey_Wood(a.pos + [600*S+box_size*2*6,-(i+1)*2*box_size]))
    master_box_list[-1].floating = True  
ticker = -1

master_gettable_list = []

for k in range(10):
    master_gettable_list.append(Fruit(a.pos + [600*S+box_size*2*k,-3*2*box_size]))


shifts = [np.array([randint(-10,10)/5,randint(-10,10)/5]) for _ in range(5)]
shifts.append(np.array([0,0]))


icon_size = 25.0
icon_spacing = 8*icon_size
display_height = icon_size*1.5
display_fruit = Fruit([display_height,display_height])
display_fruit.transform *= icon_size/display_fruit.size[0]
display_fruit.corporeal,display_fruit.solid = False,False
display_box = Box.Wood([icon_spacing+display_height,display_height])
display_box.transform *= icon_size/display_box.size[0]
display_box.corporeal,display_box.solid = False,False
display_life = Box.Box([icon_spacing*2+display_height,display_height],black,line_color = (20,20,20),line_width = 2)
display_life.transform *= icon_size/display_life.size[0]
display_life.corporeal,display_life.solid = False,False


font = pygame.font.Font('freesansbold.ttf', int(icon_size*1.7))
 
def display_score(score_list):
    
    if score_list['fruit'] > 99:
        score_list['fruit'] -= 100
        score_list['lives'] += 1
    
    display_fruit.draw(gameDisplay)
    text = font.render(str(score_list['fruit']), True, black, None)
    textRect = text.get_rect()  
    textRect.center = (display_height*2 + textRect[3]/2,display_height+icon_size*0.15) 
    gameDisplay.blit(text, textRect) 
    
    display_box.draw(gameDisplay)
    text = font.render(str(score_list['boxes']), True, black, None)
    textRect = text.get_rect()  
    textRect.center = (icon_spacing + display_height*2 + textRect[3]/2,display_height+icon_size*0.15) 
    gameDisplay.blit(text, textRect) 
    
    display_life.draw(gameDisplay)
    text = font.render(str(score_list['lives']), True, black, None)
    textRect = text.get_rect()  
    textRect.center = (icon_spacing*2 + display_height*2 + textRect[3]/2,display_height+icon_size*0.15) 
    gameDisplay.blit(text, textRect) 

            
left_key = False
right_key = False
up_key = False
down_key = False
crouch_key = False
jump_hold = False

character= Player([200*S,floor-300*S])

# Object to determine what to draw on screen
screen = Body([0,0],display_size,corporeal=True,solid=False,velocity=[0,0])   
    
death_delay = 60 # time after death level keeps tickin'
crashed = death_delay
jump_timer = 0

while True:
    
    if crashed:
        
        if crashed == death_delay:
            character= Player([200*S,floor-300*S])
            crashed = False
            box_list = copy.deepcopy(master_box_list)
            platform_list = copy.deepcopy(master_platform_list)
            gettable_list = copy.deepcopy(master_gettable_list)
            
            foreground_list = []
            score_list = {'lives':14, 'fruit': 80, 'boxes':0}
            
            
        else:    
            crashed += 1

    jump_timer -=1
    jump_key = False
    attack_key = False
    flop_key = False
    
    for event in pygame.event.get(): ### THIS IS NEEDED OR IT WILL CRASH

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                left_key = True
            elif event.key == pygame.K_RIGHT:
                right_key = True
                
            elif event.key == pygame.K_UP:
                up_key = True
            elif event.key == pygame.K_DOWN:
                down_key = True
                
            elif event.key == pygame.K_c:
                crouch_key = True
                flop_key = True
        
            elif event.key == pygame.K_x:
                jump_key = True
                jump_timer = character.jump_anticipation
                jump_hold = True
            
            elif event.key == pygame.K_z:
                attack_key = True
   
            elif event.key == pygame.K_q:
                crashed = True
                
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_key = False
            elif event.key == pygame.K_RIGHT:
                right_key = False
            elif event.key == pygame.K_c:
                crouch_key = False
            
            elif event.key == pygame.K_UP:
                up_key = False
            elif event.key == pygame.K_DOWN:
                down_key = False
            elif event.key == pygame.K_x:
                jump_hold = False

        ######################
    
    
    
    # Move every non-player object
    for bod in box_list:
        bod.move() # boxes not floating and not resting on will accelerate down
        if bod.vel[1] >0 and bod.solid:
            for bod2 in box_list+platform_list:
                if resolve_fall(bod,bod2):
                    if not isinstance(bod2,Box.Box) or bod2.vel[1] == 0:
                        bod.vel[1] = 0
                        break
                        
    for bod in platform_list:
        bod.move()
    
    if not crashed: # if character is dead, don't bother with this stuff until it's regenerated
        
        # determine character's state and velocity
        character.evolve(right_key-left_key,crouch_key,jump_key,attack_key,flop_key,jump_hold)

        character.move()
        squeeze = [[False,False],[False,False]]
        
        # sort box_list so that closest bodies interact first. 
        # this prevents incorrect interactions like diagonally breaking boxes
        box_list = sorted(box_list, key= lambda x: sum((character.pos-x.pos)**2), reverse=False)



        for k,i in enumerate(box_list+platform_list+gettable_list):

            if i.solid: # if corporeal object (not fruit)
                
                
               # break_box, bounce, 
                
                dim,gap = character.overlap_dim(i)
                
                if isinstance(character.hit_box(),Body):
                    hit = character.hit_box().overlap(i)
                else:
                    hit = False

                    
                if dim>-1: # overlap!
                   # print(dim)
                    side = 2*(character.pos[dim] > i.pos[dim]) - 1
                    converging = (character.vel[dim]-i.vel[dim])*side < 0 # if character is moving towards object


                    if isinstance(i,Box.Wood):
                        bounce = (dim == 1) and converging and (not hit)
                        if bounce:
                            i.bounces -=1
                        break_box = (bounce and (i.bounces == 0 or i.vel[1]>0)) or hit # falling bouncy boxes break
                        if bounce and not break_box:
                            if isinstance(character.resting_on,Body) or (jump_timer == character.jump_anticipation and dim == 1 and side == 1 and character.jumping > 0):
                                break_box = True
                            else:
                                if isinstance(i,Box.Bouncey_Wood):
                                    score_list['fruit'] += 1
                                wood_bounce_sound()

                        if break_box:
                            wood_break_sound()
                            
                    elif isinstance(i,Box.Metal_Wood):
                        bounce = False
                        break_box = character.flopping and hit
                        if break_box:
                            wood_break_sound()

                    elif isinstance(i,Box.Nitro):
                        break_box = True
                        bounce = False
                    elif isinstance(i,Box.Tnt):
                        break_box = hit
                        bounce = dim == 1 and converging and (not hit) # and not timer already started
                        # countdown_timer_start = (dim == 1 and converging) 
                    else:
                        break_box,bounce = False,False


                    if not break_box:  # record squeeze, remove overlap, reduce velocity in this dimension to 0
                        squeeze[dim][int(character.pos[dim] > i.pos[dim])] = i
                       
                        character.pos[dim] += side*gap
                        if converging: 
                            character.vel[dim] = 0.0
                    else:
                        score_list['boxes'] += 1
                        score_list['fruit'] += i.fruit
                        i.destroy()
                        if isinstance(i,Box.Nitro) or isinstance(i,Box.Tnt):
                            crashed = True
                            box_list.remove(i)
                            foreground_list.append(i)
                            boom_sound()
                      

                    if bounce: 
                        if not (character.flopping>0) or not break_box: # if character is flopping and box breaks, no bounce
                            character.bounce(jump_timer*jump_hold,side) 
                            # first input is jump was recently pressed and still held


                    if side == -1 and dim == 1 and not break_box and not bounce: 
                        character.is_on(i) # character above, colliding vertically, i is not broken, character not bouncing
                    
                    if dim != 1 and character.resting_on == i:
                        character.is_off()
                        
                elif hit: # if hit box overlap but no character overlap
                    if isinstance(i,Box.Box):
                        score_list['boxes'] += 1
                        score_list['fruit'] += i.fruit
                        
                        if not isinstance(i,Box.Metal) and (not isinstance(i,Box.Metal_Wood) or character.flopping>0):
                            i.destroy()

                            if isinstance(i,Box.Tnt) or isinstance(i,Box.Nitro):
                                crashed = True
                                box_list.remove(i)
                                foreground_list.append(i)
                                boom_sound()
                            else:
                                wood_break_sound()

                        
                else:
                    if character.resting_on == i:
                        character.vel += i.vel
                        character.is_off() # not on anything no more
                    if character in i.on_me:
                        i.on_me.remove(character)

            else: #non-corporeal, do not adjust velocity
    
                if isinstance(i,Fruit):
                    if character.overlap(i):
                       #i.color = tuple([i*.98 for i in list(i.color)])
                        gettable_list.remove(i)
                        fruit_sound()
                        score_list['fruit'] += 1
                        i.visible = False

        # overlaps on both sides for a given dimension, this should by design happen rarely
        if all(squeeze[1]):
            print('squeezed')
            crashed = True # dead!

        # if character completely off screen, ded.
        if not screen.overlap(character):
            crashed = True #dead!


    # Earthquake maker!
    if ticker == -1 and False:
        ticker = len(shifts)-2  
    elif ticker>=0:
        ticker -=1
    
    
    gameDisplay.fill(white)
    screen.pos = np.array([character.pos[0]-display_size[0]/2,0])
    
   
    big_list = background_list+platform_list+box_list+gettable_list
    if not crashed:
        big_list += [character]
        
    big_list += foreground_list
    
    for bod in big_list:
        
        if ticker>=0:
            bod.visual_shift(shifts[ticker]-shifts[ticker+1])
        if screen.overlap(bod):
            bod.draw(gameDisplay,screen.pos)

    display_score(score_list)
    
    
  #  sleep(0.1)
    pygame.display.flip()
    clock.tick(45)

    
pygame.quit()

KeyboardInterrupt: 

In [ ]:
x = 2
master_body_list[x].on_me,master_body_list[x]

In [3]:
foreground_list


[]

In [ ]:
    squeeze = [[],[]]

In [5]:
text_rect

NameError: name 'text_rect' is not defined

In [7]:
text = font.render(str(score_list['fruit']), True, black, None)
textRect = text.get_rect()  
textRect.center = (display_height*2.5,display_height) 

In [2]:
x = 5

In [3]:
x %=2

In [4]:
x

1